In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [2]:
import optuna

In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
alembic                          1.13.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
b

In [4]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [5]:
data=pd.read_csv("https://raw.githubusercontent.com/Chandrakant817/Admission-Prediction/main/Admission_Prediction.csv")

In [6]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,500.0,250.500000,144.481833,1.00,125.7500,250.50,375.25,500.00
GRE Score,485.0,316.558763,11.274704,290.00,308.0000,317.00,325.00,340.00
TOEFL Score,490.0,107.187755,6.112899,92.00,103.0000,107.00,112.00,120.00
University Rating,485.0,3.121649,1.146160,1.00,2.0000,3.00,4.00,5.00
SOP,500.0,3.374000,0.991004,1.00,2.5000,3.50,4.00,5.00
LOR,500.0,3.484000,0.925450,1.00,3.0000,3.50,4.00,5.00
CGPA,500.0,8.576440,0.604813,6.80,8.1275,8.56,9.04,9.92
Research,500.0,0.560000,0.496884,0.00,0.0000,1.00,1.00,1.00
Chance of Admit,500.0,0.721740,0.141140,0.34,0.6300,0.72,0.82,0.97


In [9]:
data.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [10]:
data.isnull().sum().sum()

40

In [11]:
data["GRE Score"]=data["GRE Score"].fillna(data["GRE Score"].median())
data["TOEFL Score"]=data["TOEFL Score"].fillna(data["TOEFL Score"].median())
data["University Rating"]=data["University Rating"].fillna(data["University Rating"].median())

In [12]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [13]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [14]:
X=data.drop(["Serial No.","Chance of Admit"],axis=1)

In [15]:
y=data["Chance of Admit"]

In [16]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0


In [17]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit, dtype: float64

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=25)

In [19]:
sc=StandardScaler()

In [20]:
X_train_sc=sc.fit_transform(X_train)

In [21]:
X_test_sc=sc.transform(X_test)

In [22]:
X.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          500 non-null    float64
 1   TOEFL Score        500 non-null    float64
 2   University Rating  500 non-null    float64
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
dtypes: float64(6), int64(1)
memory usage: 27.5 KB


In [24]:
X["GRE Score"]

0      337.0
1      324.0
2      317.0
3      322.0
4      314.0
       ...  
495    332.0
496    337.0
497    330.0
498    312.0
499    327.0
Name: GRE Score, Length: 500, dtype: float64

In [25]:
for col in X.columns:
    print(f"{col} {X[col].apply(lambda x: isinstance(x,str)).sum()}")

GRE Score 0
TOEFL Score 0
University Rating 0
SOP 0
LOR 0
CGPA 0
Research 0


In [28]:
X.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [29]:
y.isnull().sum()

0

In [30]:
def objective(trail,data=X,target=y):
    train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=25)
    param={
      'tree_method': 'gpu_hist',
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :300,
      'max_depth' :trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight',1,200)
      }

    model=xgb.XGBRegressor(**param)
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred=model.predict(test_x)
    mse=mean_squared_error(test_y,pred)
    return mse


In [32]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2024-01-13 07:08:38,121] A new study created in memory with name: no-name-d22887c0-14fb-441a-b512-3ddc612e788d


[0]	validation_0-rmse:0.14508
[1]	validation_0-rmse:0.14507
[2]	validation_0-rmse:0.14507
[3]	validation_0-rmse:0.14504
[4]	validation_0-rmse:0.14502
[5]	validation_0-rmse:0.14501
[6]	validation_0-rmse:0.14499
[7]	validation_0-rmse:0.14498
[8]	validation_0-rmse:0.14496
[9]	validation_0-rmse:0.14494
[10]	validation_0-rmse:0.14492
[11]	validation_0-rmse:0.14491
[12]	validation_0-rmse:0.14489
[13]	validation_0-rmse:0.14487
[14]	validation_0-rmse:0.14485
[15]	validation_0-rmse:0.14482
[16]	validation_0-rmse:0.14482
[17]	validation_0-rmse:0.14480
[18]	validation_0-rmse:0.14478
[19]	validation_0-rmse:0.14476
[20]	validation_0-rmse:0.14474
[21]	validation_0-rmse:0.14472
[22]	validation_0-rmse:0.14470
[23]	validation_0-rmse:0.14468
[24]	validation_0-rmse:0.14468
[25]	validation_0-rmse:0.14466
[26]	validation_0-rmse:0.14464
[27]	validation_0-rmse:0.14462
[28]	validation_0-rmse:0.14460
[29]	validation_0-rmse:0.14458
[30]	validation_0-rmse:0.14456
[31]	validation_0-rmse:0.14455
[32]	validation_0-

<ipython-input-30-fa88bc03800e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-30-fa88bc03800e>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:08:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[37]	validation_0-rmse:0.14446
[38]	validation_0-rmse:0.14445
[39]	validation_0-rmse:0.14444
[40]	validation_0-rmse:0.14441
[41]	validation_0-rmse:0.14440
[42]	validation_0-rmse:0.14439
[43]	validation_0-rmse:0.14437
[44]	validation_0-rmse:0.14435
[45]	validation_0-rmse:0.14433
[46]	validation_0-rmse:0.14430
[47]	validation_0-rmse:0.14428
[48]	validation_0-rmse:0.14427
[49]	validation_0-rmse:0.14424
[50]	validation_0-rmse:0.14422
[51]	validation_0-rmse:0.14420
[52]	validation_0-rmse:0.14420
[53]	validation_0-rmse:0.14418
[54]	validation_0-rmse:0.14416
[55]	validation_0-rmse:0.14414
[56]	validation_0-rmse:0.14413
[57]	validation_0-rmse:0.14411
[58]	validation_0-rmse:0.14410
[59]	validation_0-rmse:0.14408
[60]	validation_0-rmse:0.14406
[61]	validation_0-rmse:0.14404
[62]	validation_0-rmse:0.14403
[63]	validation_0-rmse:0.14400
[64]	validation_0-rmse:0.14398
[65]	validation_0-rmse:0.14397
[66]	validation_0-rmse:0.14395
[67]	validation_0-rmse:0.14394
[68]	validation_0-rmse:0.14392
[69]	val

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:08:39] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
[I 2024-01-13 07:08:39,435] Trial 0 finished with value: 0.019653520192528248 and parameters: {'lambda': 1.1043250120210755, 'alpha': 0.14021060860665216, 'colsample_bytree': 0.4, 'subsample': 0.4, 'learning_rate': 0.0003, 'max_depth': 12, 'random_state': 243123, 'min_child_weight': 54}. Best is trial 0 with value: 0.019653520192528248.


...

[0]	validation_0-rmse:0.14510
[1]	validation_0-rmse:0.14510
[2]	validation_0-rmse:0.16702
[3]	validation_0-rmse:0.16702
[4]	validation_0-rmse:0.16702
[5]	validation_0-rmse:0.75557
[6]	validation_0-rmse:0.75557
[7]	validation_0-rmse:5.32569
[8]	validation_0-rmse:5.32569
[9]	validation_0-rmse:5.32569
[10]	validation_0-rmse:5.32569
[11]	validation_0-rmse:5.32569
[12]	validation_0-rmse:36.73137
[13]	validation_0-rmse:253.89359
[14]	validation_0-rmse:253.89359
[15]	validation_0-rmse:1754.66600
[16]	validation_0-rmse:1754.66600
[17]	validation_0-rmse:12117.82220
[18]	validation_0-rmse:83685.89474
[19]	validation_0-rmse:577936.78530
[20]	validation_0-rmse:577936.78530
[21]	validation_0-rmse:577936.78530
[22]	validation_0-rmse:577936.78530
[23]	validation_0-rmse:3997494.31727
[24]	validation_0-rmse:3997494.31727
[25]	validation_0-rmse:3997494.31727
[26]	validation_0-rmse:3997494.31727
[27]	validation_0-rmse:27613188.17711
[28]	validation_0-rmse:27613188.17711
[29]	validation_0-rmse:27613188.17

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:08:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[58]	validation_0-rmse:2253986024094790400.00000
[59]	validation_0-rmse:2253986024094790400.00000
[60]	validation_0-rmse:2253986024094790400.00000
[61]	validation_0-rmse:2253986024094790400.00000
[62]	validation_0-rmse:15566078905490427904.00000
[63]	validation_0-rmse:inf
[64]	validation_0-rmse:inf
[65]	validation_0-rmse:inf
[66]	validation_0-rmse:inf
[67]	validation_0-rmse:inf
[68]	validation_0-rmse:inf
[69]	validation_0-rmse:inf
[70]	validation_0-rmse:inf
[71]	validation_0-rmse:inf
[72]	validation_0-rmse:inf
[73]	validation_0-rmse:inf
[74]	validation_0-rmse:inf
[75]	validation_0-rmse:inf
[76]	validation_0-rmse:inf
[77]	validation_0-rmse:inf
[78]	validation_0-rmse:inf
[79]	validation_0-rmse:inf
[80]	validation_0-rmse:inf
[81]	validation_0-rmse:inf
[82]	validation_0-rmse:inf
[83]	validation_0-rmse:inf
[84]	validation_0-rmse:inf
[85]	validation_0-rmse:inf
[86]	validation_0-rmse:inf
[87]	validation_0-rmse:inf
[88]	validation_0-rmse:inf
[89]	validation_0-rmse:inf
[90]	validation_0-rmse:in

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:08:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
[I 2024-01-13 07:08:51,217] Trial 9 finished with value: 4.387709661511339e+73 and parameters: {'lambda': 1.3550987473502885, 'alpha': 0.00014491072577243433, 'colsample_bytree': 0.9, 'subsample': 0.3, 'learning_rate': 8, 'max_depth': 7, 'random_state': 30, 'min_child_weight': 114}. Best is trial 1 with value: 0.004509828610776746.


{'lambda': 1.1038299415263448,
 'alpha': 0.2002246174362676,
 'colsample_bytree': 0.3,
 'subsample': 0.9,
 'learning_rate': 0.008,
 'max_depth': 4,
 'random_state': 3454,
 'min_child_weight': 30}

In [34]:
best_params={'lambda': 1.1038299415263448,
 'alpha': 0.2002246174362676,
 'colsample_bytree': 0.3,
 'subsample': 0.9,
 'learning_rate': 0.008,
 'max_depth': 4,
 'random_state': 3454,
 'min_child_weight': 30}

In [ ]:
!python --version

Python 3.9.7


In [ ]:
!pip show optuna

Name: optuna
Version: 3.5.0
Summary: A hyperparameter optimization framework
Home-page: 
Author: Takuya Akiba
Author-email: 
License: MIT License


        
        Copyright (c) 2018 Preferred Networks, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
        AUTHORS OR COPYRIGHT HOLDERS BE 

In [33]:
!python --version

Python 3.10.12


In [35]:
model=xgb.XGBRegressor(**best_params)

In [42]:
model=xgb.XGBRegressor(**best_params)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

XGBRegressor(alpha=0.2002246174362676, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=1.1038299415263448,
             learning_rate=0.008, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=30, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, ...)

In [43]:
y_pred=model.predict(X_test)

In [44]:
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.5664413426399428

In [41]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8211735774938018

In [45]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)
print(full_path)

/content/logs/system.txt


In [46]:
# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

In [47]:
# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

In [48]:
# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

In [49]:
# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

In [51]:

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)


In [55]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()

CRITICAL:root:System failure: You need to handle the issue now


In [57]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)

# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

# Set the format of the logs using a Formatter
# This format includes the log timestamp, log level and log message
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(message)s'))

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)




CRITICAL:root:System failure: You need to handle the issue now


In [ ]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()